In [13]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
import numpy as np
from sklearn.utils import shuffle

In [14]:
import discord
from discord.ext import commands
import asyncio
import nest_asyncio
import numpy as np
from PIL import Image
import io

In [15]:
#검열 데이터(0)
censored_images = ["검열1.jpg", "검열2.png", "검열3.png"]

#허용범위(1)
valid_images = ["허용범위1.png", "허용범위2.png", "허용범위3.png", "허용범위4.png", "허용범위5.jpg"]

# 이미지 로드 및 전처리 함수
def load_and_process_images(image_list, label):
    data = []
    labels = []
    for img_path in image_list:
        img = load_img(img_path, target_size=(128, 128))  # 이미지 로드 및 크기 조정
        img_array = img_to_array(img) / 255.0  # 정규화 (0~1 범위)
        data.append(img_array)
        labels.append(label)  # 클래스 라벨 추가
    return np.array(data), np.array(labels)

# 데이터 로드
censored_data, censored_labels = load_and_process_images(censored_images, 0)  # 검열(0)
valid_data, valid_labels = load_and_process_images(valid_images, 1)  # 허용(1)

# 데이터 합치기
X = np.concatenate((censored_data, valid_data), axis=0)
y = np.concatenate((censored_labels, valid_labels), axis=0)

# 데이터 섞기
X, y = shuffle(X, y, random_state=42)

# CNN 모델 정의 (이진 분류)
model = Sequential([
    Conv2D(32, (3,3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(pool_size=(2,2)),
    Conv2D(64, (3,3), activation='relu'),
    MaxPooling2D(pool_size=(2,2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(1, activation='sigmoid')  # 이진 분류
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 증강 적용(데이터 불균형 해소 목적) 
datagen = ImageDataGenerator(rotation_range=20, horizontal_flip=True)
train_generator = datagen.flow(X, y, batch_size=32)

# 모델 학습
model.fit(train_generator, epochs=10)

# 모델 저장
model.save("image_filter_model.h5")


Epoch 1/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.3750 - loss: 0.7018
Epoch 2/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 244ms/step - accuracy: 0.6250 - loss: 1.8973
Epoch 3/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 243ms/step - accuracy: 0.7500 - loss: 0.6056
Epoch 4/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 248ms/step - accuracy: 0.5000 - loss: 0.6794
Epoch 5/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 257ms/step - accuracy: 0.8750 - loss: 0.5609
Epoch 6/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 322ms/step - accuracy: 0.8750 - loss: 0.4557
Epoch 7/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 276ms/step - accuracy: 0.7500 - loss: 0.4365
Epoch 8/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 0.8750 - loss: 0.3142
Epoch 9/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 271ms/step - accuracy: 0.8750 - loss: 0.2948
Epoch 10/10
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 246ms/step - accuracy: 1.0000 - loss: 0.2401


In [ ]:
nest_asyncio.apply()

# 디스코드 봇 토큰 (여기에 자신의 봇 토큰 입력)
TOKEN = 'your_token'
CHANNEL_ID = "your_channel_ID"

# 저장된 CNN 모델 로드
model = tf.keras.models.load_model("image_filter_model.h5")

# 디스코드 봇 설정
bot = commands.Bot(command_prefix="!", intents=discord.Intents.all())

# 이미지 전처리 함수 (CNN 입력 형식으로 변환)
def preprocess_image(image_bytes):
    image = Image.open(io.BytesIO(image_bytes)).resize((128, 128))
    image = np.array(image) / 255.0  # 정규화
    image = np.expand_dims(image, axis=0)  # 모델 입력 차원 맞추기
    return image

@bot.event
async def on_ready():
     print("Bot ready!")

@bot.event
async def on_message(message):
    if message.author == bot.user:
        return

    for attachment in message.attachments:
        if attachment.content_type.startswith('image'):
            image_bytes = await attachment.read()
            image = preprocess_image(image_bytes)

            prediction = model.predict(image)
            predicted_label = int(prediction[0][0] > 0.5)  # 0: 검열됨, 1: 정상

            if predicted_label == 0:  # 유해 이미지 감지
                await message.delete()
                 await asyncio.sleep(2)
                await message.channel.send(f"🚨 {message.author.mention}, 유해한 이미지가 감지되어 삭제되었습니다.")
                print(f"❌ 유해 이미지 삭제됨: {message.author}가 업로드한 이미지")

            else:
                print(f"✅ 정상 이미지: {message.author}가 업로드한 이미지")
                 await asyncio.sleep(2)


async def start_bot():
    await bot.start(TOKEN)

asyncio.run(start_bot())

Bot ready!
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 156ms/step
✅ 정상 이미지: midtermexam가 업로드한 이미지
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
❌ 유해 이미지 삭제됨: midtermexam가 업로드한 이미지
